In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")

In [ ]:
# user, place, product, video 메타데이터
user = pd.read_csv('../Data/whyout_data/user.csv') # (31177,3)
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10)
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5821,11)
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9)

In [ ]:
# 유저의 행동데이터
user_place = pd.read_csv('../Data/whyout_data/col_user_place.csv') # (31177,4697)
user_product = pd.read_csv('../Data/whyout_data/col_user_product.csv') # (31177,5821)
user_video = pd.read_csv('../Data/whyout_data/col_user_video.csv') # (31177, 3250)

In [ ]:
# 각 아이템 별 행동 데이터가 없는 유저를 삭제한 데이터
drop_user_place = pd.read_csv('../Data/whyout_data/drop_user_place.csv') # (22420,4697) 
drop_user_product = pd.read_csv('../Data/whyout_data/drop_user_product.csv') # (2994,5821)
drop_user_video = pd.read_csv('../Data/whyout_data/drop_user_video.csv') # (11067, 3250)

In [ ]:
# 각 아이템 별 행동 데이터가 없는 유저를 삭제한 데이터의 idx
drop_user_place_idx = pd.read_csv('../Data/whyout_data/drop_user_place_idx.csv') # (22420,4)
drop_user_product_idx = pd.read_csv('../Data/whyout_data/drop_user_product_idx.csv') # (2294,4)
drop_user_video_idx = pd.read_csv('../Data/whyout_data/drop_user_video_idx.csv') # (11067, 4)

In [ ]:
# 유저의 행동데이터 전체를 사용한 SGD 결과 (U x V)
full_data_sgd_place_preds = pd.read_csv('../Data/whyout_data/sgd_result/full_data/user_place_k50epochs1000.csv')
full_data_sgd_product_preds = pd.read_csv('../Data/whyout_data/sgd_result/full_data/user_product_k50epochs1000.csv')
full_data_sgd_video_preds = pd.read_csv('../Data/whyout_data/sgd_result/full_data/user_video_k50epochs1000.csv')

In [ ]:
# 각 아이템 별 행동 데이터가 없는 유저를 삭제한 데이터를 사용한 SGD 결과 (U x V)
del_data_sgd_place_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_place_k40epochs1000.csv')
del_data_sgd_product_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_product_k20epochs1000.csv')
del_data_sgd_video_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_video_k30epochs1000.csv')

In [ ]:
# 각 아이템 별 행동 데이터가 없는 유저를 삭제한 데이터를 사용한 SGD 결과의 user_latent(U)와 item_latent(V)
place_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_place_user_latent_k40epochs1000.csv')
place_item_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_place_item_latent_k40epochs1000.csv')
product_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_product_user_latent_k20epochs1000.csv')
product_item_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_product_item_latent_k20epochs1000.csv')
video_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_video_user_latent_k30epochs1000.csv')
video_item_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_video_item_latent_k30epochs1000.csv')

In [ ]:
# 사용자의 평가 유무 확인
def find_zero_indices(df):
    return df.index[df.eq(0).all(axis=1)].tolist()

def find_non_zero_indices(df):
    return df.index[~df.eq(0).all(axis=1)].tolist()

place_zero_indices = find_zero_indices(user_place)
product_zero_indices = find_zero_indices(user_product)
video_zero_indices = find_zero_indices(user_video)
common_zero_indices = set(place_zero_indices) & set(product_zero_indices) & set(video_zero_indices)

print('장소를 평가하지 않은 사용자 수:',len(place_zero_indices))
print('상품을 평가하지 않은 사용자 수:',len(product_zero_indices))
print('영상을 평가하지 않은 사용자 수:',len(video_zero_indices))
print('모두 평가하지 않은 사용자 수:',len(common_zero_indices))

place_non_zero_indices = find_non_zero_indices(user_place)
product_non_zero_indices = find_non_zero_indices(user_product)
video_non_zero_indices = find_non_zero_indices(user_video)
common_non_zero_indices = set(place_non_zero_indices) & set(product_non_zero_indices) & set(video_non_zero_indices)

print('장소를 평가한 사용자 수:',len(place_non_zero_indices))
print('상품을 평가한 사용자 수:',len(product_non_zero_indices))
print('영상을 평가한 사용자 수:',len(video_non_zero_indices))
print('모두 평가한 사용자 수:',len(common_non_zero_indices))

find = set(place_zero_indices) & set(product_non_zero_indices) & set(video_zero_indices)
print(find)

In [ ]:
# place 활동내역이 있는 user에게 place 추천
place_user_id = 10538
num_recommendations = 10

def recommend_place(sgd_place_preds, user_id, place_df, ratings_df, idx, num_recommendations):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_place_index = int(idx[idx['idx'] == user_id].iloc[:,2])
        drop_user_place_index = int(idx[idx['idx'] == user_id].iloc[:,3])
        print('user_place_index:', user_place_index)
        print('drop_user_place_index:', drop_user_place_index)

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[drop_user_place_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        print(f'이미 평가한 place 길이: {len(user_history_indices)}')
        print(f'이미 평가한 place:', user_history_indices)
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = place_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = place_df.iloc[user_history_non_indices]['idx'].tolist()

        print("이미 평가한 place idx:", non_recommendations)
        print("평가 안한 place idx:", recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = sgd_place_preds.loc[drop_user_place_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
        recommendations_result = place_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 place idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

predictions = recommend_place(del_data_sgd_place_preds, place_user_id, place, drop_user_place, drop_user_place_idx ,num_recommendations)

In [ ]:
# product 활동내역이 있는 user에게 product 추천
product_user_id = 26232
num_recommendations = 10

def recommend_product(sgd_product_preds, user_id, product_df, ratings_df, idx, num_recommendations):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_product_index = int(idx[idx['idx'] == user_id].iloc[:,2])
        drop_user_product_index = int(idx[idx['idx'] == user_id].iloc[:,3])
        print('user_product_index:', user_product_index)
        print('drop_user_product_index:', drop_user_product_index)

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[drop_user_product_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        print(f'이미 평가한 product 길이: {len(user_history_indices)}')
        print(f'이미 평가한 product:', user_history_indices)
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = product_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = product_df.iloc[user_history_non_indices]['idx'].tolist()

        print("이미 평가한 product idx:", non_recommendations)
        print("평가 안한 product idx:", recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = sgd_product_preds.loc[drop_user_product_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
        print(top_recommendations)
        recommendations_result = product_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 product idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

predictions = recommend_product(del_data_sgd_product_preds, product_user_id, product, drop_user_product, drop_user_product_idx, num_recommendations)

In [ ]:
# video 활동내역이 있는 user에게 video 추천
video_user_id = 26232
num_recommendations = 10

def recommend_video(sgd_video_preds, user_id, video_df, ratings_df, idx, num_recommendations):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_video_index = int(idx[idx['idx'] == user_id].iloc[:,2])
        drop_user_video_index = int(idx[idx['idx'] == user_id].iloc[:,3])
        print('user_video_index:', user_video_index)
        print('drop_user_video_index:', drop_user_video_index)

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[drop_user_video_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        print(f'이미 평가한 video 길이: {len(user_history_indices)}')
        print('이미 평가한 video:', user_history_indices)
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = video_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = video_df.iloc[user_history_non_indices]['idx'].tolist()

        print("이미 평가한 video idx:", non_recommendations)
        print("평가 안한 video idx:", recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = sgd_video_preds.loc[drop_user_video_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
        recommendations_result = video_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 video idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

predictions = recommend_video(del_data_sgd_video_preds, video_user_id, video, drop_user_video, drop_user_video_idx, num_recommendations)

In [ ]:
# 모든 데이터 place, product, video 추천
user_id = 7
num_recommendations = 10

def recommend_all(total_sgd_preds, user_id, total_df, ratings_df, idx, num_recommendations):
    if user_id in idx['idx'].values:
        print(f'{user_id}번 유저의 행동이 있습니다.')
        user_video_index = int(idx[idx['idx'] == user_id].iloc[:,2])
        print('user_video_index:', user_video_index)

        # 원본 평점 데이터에서 user_id에 해당하는 행을 DataFrame으로 가져온다.
        user_data = ratings_df.loc[user_video_index]

        # 사용자가 이미 평가한 상품의 인덱스를 추출
        user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]
        user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
        print(f'이미 평가한 아이템 길이: {len(user_history_indices)}')
        print('이미 평가한 아이템:', user_history_indices)
        #print(len(user_history_non_indices),user_history_non_indices)
        non_recommendations = total_df.iloc[user_history_indices]['idx'].tolist()
        recommendations = total_df.iloc[user_history_non_indices]['idx'].tolist()

        print("이미 평가한 아이템 idx:", non_recommendations)
        print("평가 안한 아이템 idx:", recommendations)

        # SGD를 통해 예측된 사용자의 평점을 기반으로 데이터 정렬
        user_predictions = total_sgd_preds.loc[user_video_index]
        user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
        sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
        top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
        recommendations_result = total_df.iloc[top_recommendations]['idx'].tolist()
        print(f"user {user_id}에게 추천해줄 {10}개 아이템 idx : {recommendations_result}")
        return recommendations_result
    else:
        print(f'{user_id}번 유저의 행동이 없습니다.')

predictions = recommend_all(del_data_sgd_place_preds, user_id, place, user_place, user, num_recommendations)

In [ ]:
# 추천 시스템 함수 백업 용도
def recommend_step1(item, sgd_video_preds, user_id, item_df, ratings_df, ratings_df_idx, num_recommendations):
    drop_user_video_index = int(ratings_df_idx[ratings_df_idx['idx'] == user_id].iloc[:,3])
    # 원본 행동 데이터에서 user_id에 해당하는 행을 가져옴
    user_data = ratings_df.loc[drop_user_video_index]
    # 유저가 평가하지 않은 아이템의 index를 가져옴
    user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
    # user_id에 해당하는 SGD 결과값을 가져온 후, 유저가 평가하지 않은 아이템의 결과값만 뽑아옴
    user_predictions = sgd_video_preds.loc[drop_user_video_index]
    user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
    # SGD 결과값이 높은 순으로 정렬
    sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
    # 상위 N개만큼 뽑아옴
    top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
    # 아이템 idx 매핑
    recommendations_result = item_df.iloc[top_recommendations]['idx'].tolist()
    #print(f"user {user_id}에게 추천해줄 {10}개 {item} idx : {recommendations_result}")
    return recommendations_result

        
def recommend_step2(dict, item, sgd_video_preds, user_id, new_user_id, item_df, ratings_df, ratings_df_idx, num_recommendations):
    #if new_user_id in ratings_df_idx['idx'].values:
    drop_user_video_index = int(ratings_df_idx[ratings_df_idx['idx'] == new_user_id].iloc[:,3])
    user_data = ratings_df.loc[drop_user_video_index]
    user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
    user_predictions = sgd_video_preds.loc[drop_user_video_index]
    user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
    sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
    top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
    recommendations_result = item_df.iloc[top_recommendations]['idx'].tolist()
    return recommendations_result
    # else:
    #     print(f'user {new_user_id}가 {item}에 대한 행동이 없음')
    #     print('else item', item)
    #     item_list = ['place', 'video', 'product']
    #     item_list.remove(item)
    #     # 유사도 계산
    #     new_user_idx = item_user_latent_cos(user_id, item, item_list, dict)
    #     print('new 유저 찾음 :', new_user_idx)
    #     # 새로운 new_user를 찾으면 recommend_step1 실행
    #     recommend_step1(item, dict[item][0], new_user_idx, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
    #     return new_user_idx

def item_user_latent_cos(user_id, original_item, item_list, dict):
    if user_id in dict[item_list[0]][3]['idx'].values:
        item = item_list[0]
    elif user_id in dict[item_list[1]][3]['idx'].values:
        item = item_list[1]
    else:
        """
        모든 아이템에 대한 행동이 없는 유저에게 추천하는 함수 추가 
        """
        print(f'user {user_id}는 모든 아이템에 대한 행동이 없음')

    print(f'2. {item} user latent에서 user {user_id}과 유사한 user 찾기')
    drop_user_place_index = int(dict[item][3][dict[item][3]['idx'] == user_id].iloc[:,3])
    cosine_sim_matrix = cosine_similarity(dict[item][4])
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=dict[item][4].index, columns=dict[item][4].index)
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_place_index]
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_place_index] = -1
    # 유사도가 높은 순으로 정렬
    sorted_user_similarities = user_similarities.sort_values(ascending=False)
    # 가장 유사한 유저 선택
    for i in range(len(sorted_user_similarities)):
        most_similar_user_id = sorted_user_similarities.index[i]
        new_user_id = int(dict[item][3][dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0])
        highest_similarity_score = sorted_user_similarities.iloc[i]
        if new_user_id in dict[original_item][3]['idx'].values:
            # new_user_idx 찾기
            new_user_id = int(dict[item][3][dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0])
            print('update new_user_id:', new_user_id)
            break
        else:
            print(f'{new_user_id}가 {original_item}에 대한 행동이 없음')
            
    print(f'user {user_id}과 가장 유사한 user : {new_user_id}, cos : {highest_similarity_score}')
    return new_user_id

def recommendation_system(user_id, item, item_list, dict, num_recommendations):
    if user_id in dict[item][3]['idx'].values:
        recomm_list = recommend_step1(item, dict[item][0], user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
        print(f"user {user_id}에게 추천해줄 {10}개 {item} idx : {recomm_list}")
    else:
        print(f'1. user {user_id}는 {item}에 대한 행동내역이 없음')
        # 아이템 리스트에서 행동이 없는 아이템 제거
        item_list.remove(item)
        new_user_id = item_user_latent_cos(user_id, item, item_list, dict)
        next_item = item_list[0]
        print(f'user {new_user_id}에게 {item} recommend_step2 시작')
        recom_list2 = recommend_step2(dict, item, dict[item][0], user_id, new_user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
        print(f"user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recom_list2}")

In [ ]:
def recommend_step1(item, sgd_video_preds, user_id, item_df, ratings_df, ratings_df_idx, num_recommendations):
    drop_user_video_index = int(ratings_df_idx[ratings_df_idx['idx'] == user_id].iloc[:,3])
    # 원본 행동 데이터에서 user_id에 해당하는 행을 가져옴
    user_data = ratings_df.loc[drop_user_video_index]
    # 유저가 평가하지 않은 아이템의 index를 가져옴
    user_history_non_indices = [int(i) for i in user_data[user_data <= 0].index.tolist()]
    # user_id에 해당하는 SGD 결과값을 가져온 후, 유저가 평가하지 않은 아이템의 결과값만 뽑아옴
    user_predictions = sgd_video_preds.loc[drop_user_video_index]
    user_predictions_filtered = user_predictions.iloc[user_history_non_indices]
    # SGD 결과값이 높은 순으로 정렬
    sorted_predictions = user_predictions_filtered.sort_values(ascending=False)
    # 상위 N개만큼 뽑아옴
    top_recommendations = sorted_predictions.index.tolist()[:num_recommendations]
    # 아이템 idx 매핑
    recommendations_result = item_df.iloc[top_recommendations]['idx'].tolist()
    #print(f"user {user_id}에게 추천해줄 {10}개 {item} idx : {recommendations_result}")
    return recommendations_result

def item_user_latent_cos(user_id, original_item, item_list, dict):
    print('유사도 선택시 item_list:', item_list)
    if user_id in dict[item_list[0]][3]['idx'].values:
        item = item_list[0]
        print(f'user {user_id}는 {item}에 대한 행동이 존재함')
    elif user_id in dict[item_list[1]][3]['idx'].values:
        item = item_list[1]
        print(f'user {user_id}는 {item}에 대한 행동이 존재함')
    else:
        """
        모든 아이템에 대한 행동이 없는 유저에게 추천하는 함수 추가 
        """
        print(f'user {user_id}는 모든 아이템에 대한 행동이 없음')

    print(f'2. {item} user latent에서 user {user_id}과 유사한 user 찾기')
    drop_user_place_index = int(dict[item][3][dict[item][3]['idx'] == user_id].iloc[:,3])
    cosine_sim_matrix = cosine_similarity(dict[item][4])
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=dict[item][4].index, columns=dict[item][4].index)
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_place_index]
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_place_index] = -1
    # 유사도가 높은 순으로 정렬
    sorted_user_similarities = user_similarities.sort_values(ascending=False)
    # 유사도가 높은 유저를 순서대로 기존의 추천하려는 아이템에 행동이 있는지 확인
    for i in range(len(sorted_user_similarities)):
        most_similar_user_id = sorted_user_similarities.index[i]
        new_user_id = int(dict[item][3][dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0])
        highest_similarity_score = sorted_user_similarities.iloc[i]
        
        if new_user_id in dict[original_item][3]['idx'].values:
            # new_user_idx 찾기
            new_user_id = int(dict[item][3][dict[item][3].iloc[:,3] == most_similar_user_id].iloc[:,0])
            print('3. update new_user_id:', new_user_id)
            break
        else:
            print(f'{new_user_id}가 {original_item}에 대한 행동이 없음')
    print(f'4. user {user_id}과 가장 유사한 user : {new_user_id}, cos : {highest_similarity_score}')
    return new_user_id


def recommendation_system(user_id, item, item_list, dict, num_recommendations):
    if user_id in dict[item][3]['idx'].values:
        recomm_list = recommend_step1(item, dict[item][0], user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
        print(f"1. user {user_id}에게 추천해줄 {10}개 {item} idx : {recomm_list}")
    else:
        print(f'1. user {user_id}는 {item}에 대한 행동내역이 없음')
        # 아이템 리스트에서 행동이 없는 아이템 제거
        item_list.remove(item)
        new_user_id = item_user_latent_cos(user_id, item, item_list, dict)
        user_id = new_user_id
        print(f'5. user {new_user_id}에게 {item} recommend_step2 시작')
        recom_list2 = recommend_step1(item, dict[item][0], user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
        print(f"6. user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recom_list2}")

In [ ]:
user_id = 26232
num_recommendations = 10
dict = { 'place' : [del_data_sgd_place_preds, place, drop_user_place, drop_user_place_idx, place_user_latent],
         'video' : [del_data_sgd_video_preds, video, drop_user_video, drop_user_video_idx, video_user_latent],
         'product' : [del_data_sgd_product_preds, product, drop_user_product, drop_user_product_idx, product_user_latent]}
item = 'place'
item_list = list(dict.keys())

# user 3은 video만 있음, place, product 없음
# user 20001은 place만 있음, product, video 없음
# user 26232는 product만 있음, place, video 없음 
a = recommendation_system(user_id, item, item_list, dict, num_recommendations)

In [ ]:
def place_user_latent_cos(user_id, data, idx):
    drop_user_place_index = int(idx[idx['idx'] == user_id].iloc[:,3])

    cosine_sim_matrix = cosine_similarity(data)
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=data.index, columns=data.index)
    
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_place_index]
    
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_place_index] = -1
    
    # user_id와 가장 유사한 유저 선택
    most_similar_user_id = user_similarities.idxmax()
    highest_similarity_score = user_similarities.max()

    # new_user_idx 찾기
    new_user_id = int(idx[idx.iloc[:,3] == most_similar_user_id].iloc[:,0])
    print(f'The user most similar to user {user_id} is user {new_user_id} with a similarity score of {highest_similarity_score}')

    return new_user_id

similar_user_id = place_user_latent_cos(3, place_user_latent, drop_user_place_idx)

In [ ]:
def product_user_latent_cos(user_id, data, idx):
    drop_user_product_index = int(idx[idx['idx'] == user_id].iloc[:,3])

    cosine_sim_matrix = cosine_similarity(data)
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=data.index, columns=data.index)
    
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_product_index]
    
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_product_index] = -1

    # user_id와 가장 유사한 유저 선택
    most_similar_user_id = user_similarities.idxmax()
    highest_similarity_score = user_similarities.max()

    # new_user_idx 찾기
    new_user_id = int(idx[idx.iloc[:,3] == most_similar_user_id].iloc[:,0])
    print(f'The user most similar to user {user_id} is user {new_user_id} with a similarity score of {highest_similarity_score}')

    return new_user_id

similar_user_id = product_user_latent_cos(26232, product_user_latent, drop_user_product_idx)

In [ ]:
def video_user_latent_cos(user_id, data, idx):
    drop_user_video_index = int(idx[idx['idx'] == user_id].iloc[:,3])

    cosine_sim_matrix = cosine_similarity(data)
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=data.index, columns=data.index)
    
    # user_id의 score를 가져옴
    user_similarities = cosine_sim_df.loc[drop_user_video_index]
    
    # user_id를 선택하지 않도록 -1을 해줌
    user_similarities[drop_user_video_index] = -1
    
    # user_id와 가장 유사한 유저 선택
    most_similar_user_id = user_similarities.idxmax()
    highest_similarity_score = user_similarities.max()

    # new_user_idx 찾기
    new_user_id = int(idx[idx.iloc[:,3] == most_similar_user_id].iloc[:,0])
    print(f'The user most similar to user {new_user_id} is user {similar_user_id} with a similarity score of {highest_similarity_score}')

    return new_user_id

input_user_id = 3
similar_user_id = video_user_latent_cos(input_user_id, video_user_latent, drop_user_video_idx)